In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd 
import os

In [6]:
def set_dir_path():
    """Used at runtime to ensure this file is run in the correct directory - 
    i.e. is run in the MySQLApp folder, not anywhere else.
    """
    # curr_path = os.path.dirname(os.path.abspath())
    # os.chdir(curr_path)
    # print(curr_path, os.getcwd())
os.chdir()

'c:\\Users\\Cerea\\OneDrive\\Documents\\GitHub\\pathways-planner'

In [2]:
with open("workday-data/Spring 2025 last.html", "r") as file:
    content = file.read()

soup = BeautifulSoup(content, "html.parser")

In [10]:
# Manually
instructor = soup.find(attrs={"id": lambda x: x and x.startswith("gwt-uid-")})
instructor_id   = int(instructor.attrs["id"].split('-')[-1])
course_id       = instructor_id-2
room_id         = instructor_id-1
course  = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % course_id})
room    = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % room_id})

course_result     = course.text.split('-')
room_result       = room.text.split(' | ')
instructor_result = instructor.text.strip().split(' | ')

courseCode, courseSection, courseTitle = [x.strip() for x in course_result] if len(course_result) == 3 else (course.text, None, None)
roomId, days, time                     = room_result                        if len(room_result)   == 3 else (room.text, None, None)
cStatus, insName, cMode, cFill         = [x.strip() for x in instructor_result] if len(instructor_result) == 4 else (instructor.text, None, None, None)

daf = pd.DataFrame(data=[courseCode, courseSection, courseTitle, roomId, days, time, cStatus, insName, cMode, cFill])


In [114]:
courses = soup.find_all(attrs={"id": lambda x: x and x.startswith("promptOption-gwt-uid-")})
ex = courses[0]
ex.attrs["id"].split("-")
course = int(ex.attrs["id"].split("-")[-1])
course

148

In [ ]:
i = soup.find(attrs={"id": lambda x: x and x.startswith("gwt-uid-119")})

i_id = int(i.attrs["id"].split("-")[-1])
c_id = i_id-2
r_id = i_id-1

c  = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % c_id})
r    = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % r_id})

i

In [3]:
errors = []
data   = []

instructors = soup.find_all(attrs={"id": lambda x: x and x.startswith("gwt-uid-")})

for instructor in tqdm(instructors):
    try:
        instructor_id   = int(instructor.attrs["id"].split('-')[-1])
        
        course_id       = instructor_id-2
        room_id         = instructor_id-1
        course  = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % course_id})

 

        if course != None:
            room    = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % room_id}) 
            course_result     = course.text.split('-')
            room_result       = room.text.split(' | ')
        else:
            course = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % room_id}) 
            # print(room_id, course)
            room = "empty | empty | empty"
            course_result = course.text.split("-")
            room_result = room.split(" | ")

        if len(room_result) == 3:
            roomId, days, time                     = room_result
        elif len(room_result) > 3:
            roomId, days, time                     = (room_result[0], room_result[1], room_result[2])   
        # no roomId
        elif len(room_result) == 2:
            roomId, days, time                     = ("empty", room_result[0], room_result[1])    
        else:
            roomId, days, time                     = (room_result, "empty", "empty")

        course_result = [x.strip() for x in course_result]
        if len(course_result) <= 3:
        # i think this may just be an issue with that one class AST 470-TU
            courseCode, courseSection, courseTitle = course_result 
        elif len(course_result) > 3:
            # in case the title has dashes in it
            courseCode, courseSection, courseTitle = (course_result[0], course_result[1], "-".join((course_result[2:])))

        instructor_result = instructor.text.strip().split(' | ')
        if len(instructor_result) == 4:
            cStatus, insName, cMode, cFill         = [x.strip() for x in instructor_result]
        else:
            cStatus, cMode, cFill                  = [x.strip() for x in instructor_result]
            insName = "empty"
        
        data.append([instructor_id, courseCode, courseSection, courseTitle, roomId, days, time, cStatus, insName, cMode, cFill])

    except Exception as e:
        print(e)
        errors.append({int(instructor.attrs["id"].split('-')[-1]): instructor})
        continue

len(data), len(errors)

  5%|▍         | 55/1121 [00:02<01:01, 17.43it/s]

not enough values to unpack (expected 3, got 1)


100%|██████████| 1121/1121 [04:27<00:00,  4.19it/s]


(1120, 1)

In [278]:
# what seems to be happening is these weird x/0 classes, that only have two ids - they have no section details
    # basically: section details=(empty)
errors

[{171: <span class="gwt-InlineLabel WJWF WIVF" data-automation-id="compositeSubHeaderOne" id="gwt-uid-171" title="Closed   |   Victoria Montrose   |   In-Person   |   1/0">Closed   |   Victoria Montrose   |   In-Person   |   1/0</span>}]

In [4]:
df = pd.DataFrame(data)
df.columns = ['Instructor-gwt Id','CourseCode', 'Section', 'CourseTitle', 'RoomId', 'Days', 'Time', 'Status', 'InstructorName', 'Mode', 'Fill']
# df.to_csv("Spring25.csv", index=False)

In [258]:
print(set(df["CourseCode"]))

{'MUS 524A', 'MUS 544C', 'PHL 507', 'PSY 485', 'BIO 322', 'MUS 551E', 'REL 238', 'FYW 1111', 'MUS 552E', 'ENG 223', 'SCI 201', 'MUS 543B', 'SCI 220', 'PHL 215', 'MUS 532E', 'PHL 310', 'MUS 543E', 'COM 461', 'BIO 426', 'MUS 512D', 'FYW 1327', 'MUS 525E', 'BIO 401', 'MUS 351A', 'MUS 545', 'EDU 435', 'MUS 327', 'ECN 475', 'MUS 535C', 'MUS 521D', 'MUS 548E', 'CHN 202', 'MUS 552A', 'REL 376', 'MUS 524E', 'CSC 372', 'ANT 205', 'FYW 1206', 'MUS 547', 'ENG 270', 'MUS 505', 'MUS 543C', 'MUS 551C', 'MSL 402', 'CHM 120', 'MTH 160', 'MTH 151', 'MUS 548B', 'MUS 525D', 'CHM 340', 'PHL 270', 'BIO 222', 'THA 501', 'ENG 215', 'ART 246', 'CHM 101', 'HST 278', 'MUS 572D', 'MUS 549E', 'FYW 1266', 'PSY 328', 'MUS 065', 'POL 101', 'ART 133', 'ECN 345', 'MUS 115', 'MUS 544A', 'ART 330', 'PSY 219', 'BUS 270', 'REL 218', 'SOC 101', 'MUS 521A_', 'MUS 220', 'MUS 531D', 'PHY 312', 'ENG 212', 'MUS 526A', 'BIO 102', 'COM 102', 'MUS 212C', 'MUS 545D', 'MUS 551A', 'POL 259', 'MUS 571E', 'MTH 120', 'MUS 351D', 'MUS 54

In [280]:
course_codes = df["CourseCode"].apply(lambda x: x and x.split(" ")[0])
course_codes.unique()

array(['ACC', 'AFS', 'ANT', 'ART', 'AST', 'BIO', 'BUS', 'CHM', 'CHN',
       'CLS', 'COM', 'CSC', 'DAN', 'ECN', 'EDU', 'EES', 'ENG', 'EST',
       'FRN', 'FST', 'FYW', 'GRK', 'GRM', 'HSC', 'HST', 'HUM', 'IGD',
       'JPN', 'LAS', 'LNG', 'LTN', 'MEM', 'MES', 'MLL', 'MSL', 'MTH',
       'MUS', 'NSC', 'PHL', 'PHY', 'POL', 'PSY', 'PTH', 'PVS', 'REL',
       'SCI', 'SOC', 'SPN', 'SUS', 'THA', 'UNV', 'WGS'], dtype=object)

In [ ]:
# Original Exceptions
# AST 470-TU (falls under "2025 Spring Undergraduate")
# BIO 423-TSL: has a days and time, but no roomId 
# CHM 240-01: again, days and time no roomId
# CHN 402-01: same issue
# CLS 216-01: CourseCode is CLS-216-01 + - fullName, section and courseTitle empty
# CSC 121-02L: again, days and time, no roomId
# EDU 331 and EDU-332: coursecode encompasses section and Title
# ENG 363: again, days and time no roomId
# IGD 101: wtf, it has four elements: roomId, days, time, and date?
# MUS 101-03: like IGD-101
# MUS 115-CBE: CourseCode encompassing
# MUS 412-01: again, days and time no roomId
# POL 884 and POL-886: coursecode encompasses section and Title
# PTH 203: like IGD-101
# SPN 215: again, days and time no roomId
# SUS 216: again, days and time no roomId
# UNV XXX: all sorts of fucked up, there is the courseCode encompassing error - there are 4 dashes, giving us 5 elements in the courseCode
# MUS generally has some weird underscores in the CourseCode (MUS 522A_), maybe just some weird thing on workday (not an issue?)

In [102]:
# so far, all have passed the test - except
# AST 470-TU (falls under "2025 Spring Undergraduate")

# 1 issues:
# AST 470-TU's werid formatting
# cs_courses = df[df['CourseCode'].apply(lambda x: x.startswith('2025 Spring Undergraduate'))]
cs_courses = df[df['CourseCode'].apply(lambda x: x.startswith('BIO'))]
cs_courses

# ids = df[df["CourseCode"]=="CSC 105"]
# ids


,Instructor-gwt Id,CourseCode,Section,CourseTitle,RoomId,Days,Time,Status,InstructorName,Mode,Fill
50,174,BIO 075,01,Seminar in Biology,PYR-143,T,11:30 AM - 12:45 PM,Closed,Greg Lewis,In-Person,12/11
51,177,BIO 075,02,Seminar in Biology,PYR-143,R,11:30 AM - 12:45 PM,Closed,Min-Ken Liao,In-Person,12/11
52,180,BIO 102,01,Biology and the Environment,PYR-126,TR,10:00 AM - 11:15 AM,Open,Travis Perry,In-Person,18/28
53,183,BIO 102,01L,Biology and the Environment,PYR-141,R,2:30 PM - 5:20 PM,Open,Travis Perry,In-Person,18/28
54,186,BIO 111,01,Foundations of Biology,PYR-028,TR,8:30 AM - 9:45 AM,Open,Wade Worthen,In-Person,23/28
...,...,...,...,...,...,...,...,...,...,...,...
110,337,BIO 510,05,Thesis,empty,empty,empty,Open,Adi Dubash,In-Person,3/0
111,339,BIO 510,06,Thesis,empty,empty,empty,Open,John Quinn,In-Person,2/0
112,341,BIO 510,07,Thesis,empty,empty,empty,Open,Victoria Turgeon,In-Person,1/0
113,343,BIO 510,08,Thesis,empty,empty,empty,Open,Min-Ken Liao,In-Person,1/0


In [329]:
df[df.index==1118]["InstructorName"]

1120

In [97]:
# get course_id - section details all share a section, not instructor
end = int(df["Instructor-gwt Id"].iloc[-1])+1
# sect_id_vals = (df["Instructor-gwt Id"]-1).values.tolist()
sect_info = (df[["Instructor-gwt Id", "RoomId", "Days", "Time"]])
sect_id = df["Instructor-gwt Id"]-1
sect_info["Section-gwt Id"] = sect_id
sect_info

# text = " | ".join(sect_info.loc[3][["RoomId","Days", "Time"]])
# selection = soup.find_all(attrs={"title": "%s" % text})
# oldSection, newSection = selection[0], selection[1]
# oldSection, newSection
error_data = []
for i in tqdm(sect_info.index):
    text = " | ".join(sect_info.loc[i][["RoomId","Days", "Time"]])
    selection = soup.find_all(attrs={"title": "%s" % text})
    if len(selection) != 2:
        # print("OOPS")
        error_data.append([i, selection])
error_data
# sect_info["Section-gwt Id"][0]+end
# section_ids = pd.DataFrame([temp], columns=["OldId"])
# # section_ids["NewId"] = section_ids["OldId"] - section_ids.iloc["OldId"]
# # section_pattern += end
# section_ids["NewId"] = section_ids["OldId"] - int(section_ids["OldId"][0])
# section_ids["NewId"] += end
# section_ids
# for x in section_ids.index:
#     section_pattern += section_ids[x] - section_ids[x+1]
#     print(section_pattern)
# end

C:\Users\Cerea\AppData\Local\Temp\ipykernel_27400\1985208350.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sect_info["Section-gwt Id"] = sect_id
100%|██████████| 1120/1120 [03:49<00:00,  4.88it/s]


[[38, []],
 [41, []],
 [43, []],
 [44, []],
 [45, []],
 [46, []],
 [87, []],
 [94, []],
 [95, []],
 [96, []],
 [97, []],
 [98, []],
 [99, []],
 [100, []],
 [101, []],
 [102, []],
 [103, []],
 [104, []],
 [105, []],
 [106, []],
 [107, []],
 [108, []],
 [109, []],
 [110, []],
 [111, []],
 [112, []],
 [113, []],
 [114, []],
 [135, []],
 [136, []],
 [137, []],
 [138, []],
 [139, []],
 [140, []],
 [141, []],
 [142, []],
 [171, []],
 [181, []],
 [182, []],
 [183, []],
 [184, []],
 [185, []],
 [186, []],
 [187, []],
 [188, []],
 [189, []],
 [190, []],
 [191, []],
 [192, []],
 [193, []],
 [194, []],
 [195, []],
 [196, []],
 [197, []],
 [198, []],
 [199, []],
 [200, []],
 [201, []],
 [202, []],
 [203, []],
 [204, []],
 [210, []],
 [243, []],
 [254, []],
 [255, []],
 [262, []],
 [280, []],
 [287, []],
 [288, []],
 [297, []],
 [298, []],
 [299, []],
 [300, []],
 [301, []],
 [316,
  [<div aria-label="RNK-152 | TR | 10:00 AM - 11:15 AM" class="gwt-Label WDRO WMPO" data-automation-id="promptOption" 

In [100]:
sect_info.loc[38]

Instructor-gwt Id      139
RoomId               empty
Days                 empty
Time                 empty
Section-gwt Id         138
Name: 38, dtype: object

In [ ]:
# seems like the narrowest thing that grabs per class is <div class="WGVF WEVF">
    # class=WAOF/WCOF get us the info for "Section Details"
    # class= corresponds to schedule info  (Specifically, gwt-Label WDRO WMPO)

In [419]:
num = 3025
idx = section_ids[section_ids["OldId"]==97].index
section_ids["NewId"][idx]


24    3097
Name: NewId, dtype: int64

In [21]:
id = 3097
cs_idx = section_ids[section_ids["NewId"]==id].index
old_id = section_ids["OldId"][cs_idx]
old_section_info = df[df["Instructor-gwt Id"]==old_id.iloc[0]+1][["RoomId","Days","Time"]].values.tolist()[0]
old_section_info

['RAB-PTS', 'TR', '11:30 AM - 1:20 PM']

In [ ]:
# what we need to do is find the corresponding OldId and NewId, then count up until the next item in the NewId increment is another schedule


In [ ]:
# # Now, to grab "Section Details"
# # with open("workday-data/Spring25-2.htm", "r") as file:
# #     content = file.read()

# # soup = BeautifulSoup(content, "html.parser")
# details = []
# errors_details = []
# new_ids = section_ids["NewId"]

# for id in tqdm(new_ids):
#     try:
#         section         = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % id})
#         class_type_id   = id+1
#         prq_id          = id+2

#         if (class_type_id) in new_ids or (prq_id in new_ids):
#             print("ERROR")
#             raise Exception("shallow")
        
#         class_type      = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % class_type_id})
#         prq             = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % prq_id})

#         # if course != None:
#         #     room    = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % room_id}) 
#         #     course_result     = course.text.split('-')
#         #     room_result       = room.text.split(' | ')
#         # else:
#         #     course = soup.find(attrs={"id": "promptOption-gwt-uid-%s" % room_id}) 
#         #     # print(room_id, course)
#         #     room = "empty | empty | empty"
#         #     course_result = course.text.split("-")
#         #     room_result = room.split(" | ")

#         section_result = section.text.split(" | ")
#         class_result = class_type.text
#         prq_result = prq.text.split(" - ")

#         if len(section_result) == 3:
#             roomId, days, time                     = section_result
#         elif len(section_result) > 3:
#             roomId, days, time                     = (section_result[0], section_result[1], section_result[2])   
#         # no roomId
#         elif len(section_result) == 2:
#             roomId, days, time                     = ("empty", section_result[0], section_result[1])    
#         else:
#             roomId, days, time                     = (section_result, "empty", "empty")

#         cs_idx = section_ids[section_ids["NewId"]==id].index
#         old_id = section_ids["OldId"][cs_idx]
#         old_section_info = df[df["Instructor-gwt Id"]==old_id.iloc[0]+1][["RoomId","Days","Time"]].values.tolist()[0]
#         # old_section_info = old_section_info.values

#         # df[df["Instructor-gwt Id"]==id+1-end]
#         # print(type(old_section_info.tolist()))
#         if [roomId, days, time] != old_section_info:
#             ws_bool = True
#         else:
#             ws_bool = False



#         structure = class_result
#         prqs = " ".join(prq_result[1:])
        
#         details.append([id, roomId, days, time, structure, prqs, ws_bool])

#     except Exception as e:
#         print(e)
#         errors_details.append({id, section})
#         continue


100%|██████████| 1120/1120 [04:11<00:00,  4.45it/s]


In [29]:
len(details), len(errors_details)

(1120, 0)

In [28]:
details_df = pd.DataFrame(details, columns=["CourseId","RoomId","Days","Time","Structure","PRQS", "Wrong Section?"])
details_df[details_df["Wrong Section?"]==True]

,CourseId,RoomId,Days,Time,Structure,PRQS,Wrong Section?
10,3055,[GER - Global Awareness :: WC (World Cultures)],empty,empty,IDS :: Anthropology,,True
11,3058,[PRQ - Instructor Permission Required],empty,empty,FUR-204 | TR | 1:00 PM - 2:15 PM,CORE :: HB (Empirical Study of Human Behavior),True
12,3061,"[GER - Global Awareness :: IEJ (Identities, Eq...",empty,empty,GER - Global Awareness :: WC (World Cultures),,True
13,3064,[Lecture],empty,empty,PRQ - Instructor Permission Required,12:20 PM,True
14,3067,[GER - CORE :: VP (Human Cultures - Visual and...,empty,empty,GER - Global Awareness :: WC (World Cultures),,True
...,...,...,...,...,...,...,...
1115,6013,[IDS :: Environmental Studies],empty,empty,"IDS :: Medicine, Health and Culture",,True
1116,6015,[IDS :: Poverty Studies],empty,empty,IDS :: Urban Studies,,True
1117,6017,[Lecture],empty,empty,PRQ - POL 101 Introduction to American Government,,True
1118,6020,[IDS :: Poverty Studies],empty,empty,IDS :: Urban Studies,,True


In [11]:
# What about grabbing the prereqs? div class="WAOF"?
poidData   = {}

poids = soup.find_all(attrs={"id": lambda x: x and x.startswith("promptOption-gwt-uid-")})

for poidtext in tqdm(poids):
    poid = int(poidtext.attrs["id"].split("-")[-1])
    # print("PoID: {}".format(poid))
    # print(poidtext)
    poidData[poid] = poidtext

# len(poidData)#, len(errors)

100%|██████████| 5884/5884 [00:00<00:00, 880868.21it/s]


In [188]:
course_lst = pd.Series(['ACC', 'AFS', 'ANT', 'ART', 'AST', '2025', 'BIO', 'BUS', 'CHM',\
       'CHN', 'CLS', 'COM', 'CSC', 'DAN', 'ECN', 'EDU', 'EES', 'ENG',\
       'EST', 'FRN', 'FST', 'FYW', 'GRK', 'GRM', 'HSC', 'HST', 'HUM',\
       'IGD', 'JPN', 'LAS', 'LNG', 'LTN', 'MEM', 'MES', 'MLL', 'MSL',\
       'MTH', 'MUS', 'NSC', 'PHL', 'PHY', 'POL', 'PSY', 'PTH', 'PVS',\
       'REL', 'SCI', 'SOC', 'SPN', 'SUS', 'THA', 'UNV', 'WGS'])

In [256]:
df.tail(10)#[df["CourseCode"] == "ACC 120"]

,Instructor-gwt Id,CourseCode,Section,CourseTitle,RoomId,Days,Time,Status,InstructorName,Mode,Fill
1110,2984,UNV SACI,01,CIMBA-Northern Italy,empty,empty,empty,Open,empty,In-Person,1/0
1111,2986,UNV SADN,01,Danish Institute for Study Abroad-Denmark,empty,empty,empty,Open,empty,In-Person,5/0
1112,2988,UNV SASW,01,Danish Institute for Study Abroad-Sweden,empty,empty,empty,Open,empty,In-Person,1/0
1113,2990,UNV SAUO,01,Arcadia Abroad: New Zealand University of Otago,empty,empty,empty,Open,empty,In-Person,6/0
1114,2992,UNV TSBZ,01,Belize-Travel Study,empty,empty,empty,Open,"Ashley Morris, Dennis Haney",In-Person,0/0
1115,2994,UNV TSGR,01,Germany-Travel Study,empty,empty,empty,Open,empty,In-Person,17/0
1116,2996,UNV TSIT,01,Italy-Travel Study,empty,empty,empty,Open,empty,In-Person,0/0
1117,2998,UNV TSLN,01,England-Furman in London,empty,empty,empty,Open,empty,In-Person,16/0
1118,3001,WGS 101,01,Issues in Women's Gender and Sexuality Studies,FUR-108,TR,1:00 PM - 2:15 PM,Closed,Kathleen Casey,In-Person,15/15
1119,3004,WGS 250,01,Introduction to Queer Theory and Sexuality Stu...,FUR-108,TR,2:30 PM - 3:45 PM,Open,Nick Radel,In-Person,11/18


In [254]:
# increments of 3
# so it starts at ACC 311-01 (3010)
# Then goes to ACC-120-01 (3013)
# ACC-120-02
# ACC-120-03
# ACC-120-04
# skips past ACC-311-01 to ACC-313-01
room_idx = int(df[df["Instructor-gwt Id"] == 33]["Instructor-gwt Id"] - 1)
room_idx
index = 3005 + room_idx
poidData[index], poidData[index+1], poidData[index+2]

C:\Users\Cerea\AppData\Local\Temp\ipykernel_23152\4271379224.py:8: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  room_idx = int(df[df["Instructor-gwt Id"] == 33]["Instructor-gwt Id"] - 1)


(<div aria-label="FUR-207 | MWF | 9:30 AM - 10:20 AM" class="gwt-Label WDRO WMPO" data-automation-id="promptOption" data-automation-label="FUR-207 | MWF | 9:30 AM - 10:20 AM" id="promptOption-gwt-uid-3037" role="link" tabindex="0" title="FUR-207 | MWF | 9:30 AM - 10:20 AM">FUR-207 | MWF | 9:30 AM - 10:20 AM</div>,
 <div aria-label="GER - CORE :: HB (Empirical Study of Human Behavior)" class="gwt-Label WDRO WMPO" data-automation-id="promptOption" data-automation-label="GER - CORE :: HB (Empirical Study of Human Behavior)" id="promptOption-gwt-uid-3038" title="GER - CORE :: HB (Empirical Study of Human Behavior)">GER - CORE :: HB (Empirical Study of Human Behavior)</div>,
 <div aria-label="GER - Global Awareness :: IEJ (Identities, Equity, and Justice)" class="gwt-Label WDRO WMPO" data-automation-id="promptOption" data-automation-label="GER - Global Awareness :: IEJ (Identities, Equity, and Justice)" id="promptOption-gwt-uid-3039" title="GER - Global Awareness :: IEJ (Identities, Equity,

In [199]:
poiddf = pd.DataFrame(poidData.items(), columns=["poid", "val"])
sort_poiddf = poiddf.sort_values(by="poid")
fixed = sort_poiddf["val"].apply(lambda x: "".join(x))
sort_poiddf["val"] = fixed
# sort_poiddf[sort_poiddf["val"].apply(lambda x: x not in df["CourseCode"])]
# sort_poiddf[sort_poiddf["val"].apply(lambda x: x.startswith(""))]
# sort_poiddf.tail(50)
no_sched = sort_poiddf["val"].apply(lambda x: False if x.find("|")!=-1 else True)
# no_sched = no_sched.drop(index=[no_sched["val"]==""].index)
coursee = sort_poiddf[no_sched]
# for thing in coursee["val"]:
#     three = thing
# coursee[(coursee["val"]!="Independent Study") & (coursee["val"]!="Laboratory") & (coursee["val"]!="Lecture") &\
        # (coursee["val"].apply(lambda x: False if course_lst.find(x) else True))]

# for val in coursee["val"][1500:]:
#     print(val)
# dropped = sort_poiddf.drop(index=no_sched.index)
# dropped
# sort_poiddf
# only 472 GERS, 639 PRQS
# 1121 classes
# 441 IDS
# Independent study 102, Lab 106, Lecture 622
# with no schedules, we have 4368
# sum((472, 1121, 639, 441, 106, 102, 622))

# PROMPT OPTIONS 
elements[4]

# skip 2 results 

# even ones 
# course title 
# course code 

# odd ones 
# Time, Location, Days of the week

In [13]:
len(promptOptions), len(inlineLabels)

(1880, 1183)

<div aria-label="ACC 120-02 - Principles of Financial and Managerial Accounting" class="gwt-Label WDRO WMPO" data-automation-id="promptOption" data-automation-label="ACC 120-02 - Principles of Financial and Managerial Accounting" id="promptOption-gwt-uid-7" role="link" tabindex="0" title="ACC 120-02 - Principles of Financial and Managerial Accounting">ACC 120-02 - Principles of Financial and Managerial Accounting</div>